In [ ]:
#Objective: If something was passing in mainline.. but failing in 'next', want to catch those commits causing regressions.

#Note: Not dealing with situations where we don't have data for last 2 days.

In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [ ]:
#Input Parameters
job1 = 'mainline'
job2 = 'next'
date_range = 10
threshold = 0.01
last_days=2

In [ ]:
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"

def invoke(job_name, date_range_loc):
    headers = {"Authorization":kernelci_api_key.getkernelcikey()}
    params = {
            "job": job_name,
            "date_range": date_range_loc
        }
    url = urljoin(BACKEND_URL, "/boot")
    response = requests.get(url, params=params, headers=headers)
    return response.content

def getDfBoots(job_name, date_range_loc):
    content = invoke(job_name, date_range_loc)
    contentjs = json.loads(content)
    df = json_normalize(contentjs['result'])
    
    #Select only columns we care about for boot pass/fail analysis
    df2 = df[['_id.$oid',u'arch',u'board',u'board_instance',
           u'created_on.$date',u'defconfig',u'dtb',
           u'git_describe',u'lab_name',u'mach',u'status']]

    #convert created_on to datetime and use as index
    df2['created_on'] = pd.to_datetime(df2['created_on.$date'],unit='ms')
    df2 = df2.set_index('created_on')
    df2 = df2.drop('created_on.$date',axis=1)

    #Consider only Pass and Fail
    df2 = df2[df2.status.isin(['PASS','FAIL'])]
    df2['status_fl'] = df2.status.map({'PASS':1,'FAIL':0})

    #Sort index of dates
    df2 = df2.sort_index()
    del df
    
    #Find Boot pass percentage and number of boots for all days
    df3 = df2.groupby(['defconfig','board']).mean()
    df3.columns = [job_name+'_st_L']
    df3[job_name+'_c_L'] = df2.groupby(['defconfig','board']).count().status_fl
    
    #Find Boot pass percentage and number of boots for last 'last_days' days
    df4 = df2[(df2.index[-1] - datetime.timedelta(last_days)):df2.index[-1]].groupby(['defconfig','board']).mean()
    df4.columns = [job_name+'_st_S']
    df4[job_name+'_c_S'] = df2[(df2.index[-1] - datetime.timedelta(last_days)):df2.index[-1]].groupby(['defconfig','board']).count().status_fl
      
    return (df2,df3.join(df4,how='inner'))

In [ ]:
#Plot two overlapping defconfig-board with interaction
from bokeh.plotting import figure, output_file, output_notebook,show, GridPlot 
from bokeh.models import ColumnDataSource, Circle, HoverTool,CustomJS

def retSimpleLists(df, defconfig, board, name):
    df2 = df[(df.defconfig == defconfig) & (df.board == board)]
    (x,y,k) = (df2.index.to_series(), df2['status_fl'],df2['git_describe'])
    ts = pd.to_datetime(x.values)
    z = ts.strftime('%Y.%m.%d')
    n = []
    for i in range(0,len(x)):
        n.append(name)
    return (x,y,k,z,n)

def retInteractiveFig(x,y,k,z,color,title):
    p = figure(width=600,height=300,x_axis_type='datetime', title=title)
    p.line(x,y,line_dash="4 4", line_width=2,color=color)
    source = ColumnDataSource({'x':x,'y':y,'z':z,'k':k})
    
    invisible_circle = Circle(x='x',y='y', fill_color=color, fill_alpha=0.1, line_color=None, size=10)
    visible_circle = Circle(x='x',y='y', fill_color='firebrick', fill_alpha=0.5, line_color=None, size=10)
    
    cr = p.add_glyph(source, invisible_circle, selection_glyph=visible_circle, 
                      nonselection_glyph=invisible_circle)
    
    #Add hover tool, that selects the circle
    # Add a hover tool, that selects the circle
    code = "source.set('selected', cb_data['index']);"

    callback = CustomJS(args={'source': source}, code=code)

    p.add_tools(HoverTool(
                tooltips=[
                            ("d", "@z"),
                            ("stat", "@y"),
                            ('gd', "@k")
                        ], 
                callback=callback, 
                renderers=[cr], 
                mode='vline'))
    return p

def draw_interactive_2(df1, df2, defconfig, board):
    #Reduce data to given defconfig-board combo
    x1,y1,k1,z1,n1 = retSimpleLists(df1, defconfig, board,'mainline')
    x2,y2,k2,z2,n2 = retSimpleLists(df2, defconfig, board,'next')

    output_notebook()
    
    #draw line,circles for df1 (Eg: mainline)
    p1 = retInteractiveFig(x1,y1,k1,z1,'blue','mainline')
    p2 = retInteractiveFig(x2,y2,k2,z2,'red','next')
    show(p1)
    show(p2)
    #gp = GridPlot(children=[p1,p2])
    #show(gp)

In [ ]:
df_mainline,df_mainline_sum = getDfBoots('mainline', date_range)

In [ ]:
df_next,df_next_sum = getDfBoots('next', date_range)

In [ ]:
df = df_mainline_sum.join(df_next_sum, how='inner')[['mainline_st_S','next_st_S']]
df[(df.mainline_st_S - df.next_st_S) > threshold]

In [ ]:
draw_interactive_2(df_mainline,df_next,'multi_v7_defconfig', 'qcom-msm8974-sony-xperia-honami')

In [ ]:
defconfig = 'multi_v7_defconfig'
board = 'qcom-msm8974-sony-xperia-honami'
x1,y1,k1,z1,n1 = retSimpleLists(df_mainline, defconfig, board, 'mainline')
x2,y2,k2,z2,n2 = retSimpleLists(df_next, defconfig, board, 'next')

In [ ]:
#Plot above with interaction
from bokeh.plotting import figure, output_file, output_notebook,show
from bokeh.models import ColumnDataSource, Circle, Triangle, HoverTool,CustomJS,BoxSelectTool

p = figure(width=600,height=300,x_axis_type='datetime', title='Vish')
#p.line(x1,y1,line_dash="4 4", line_width=2,color='blue')
p.line(x2,y2,line_dash="4 4", line_width=2,color='green')

source1 = ColumnDataSource({'x1':x1,'y1':y1,'z1':z1,'k1':k1,'n1':n1})
source2 = ColumnDataSource({'x2':x1,'y2':y2,'z2':z2,'k2':k2,'n2':n2})

mainline_glyph = Circle(x='x1',y='y1', fill_color='blue',fill_alpha=0.3,line_color=None, size=10)
next_glyph = Circle(x='x2',y='y2', fill_color='green',fill_alpha=0.3,line_color=None, size=10)

mainline_a = p.add_glyph(source1, mainline_glyph)
next_a = p.add_glyph(source2, next_glyph)

code = "source.set('selected', cb_data['index']);"
callback = CustomJS(args={'source1': source1}, code=code)

mainline_ht = HoverTool(
                tooltips=[("n", "@n1"),("d", "@z1"),("stat", "@y1"),('gd', "@k1")], 
                callback=callback, 
                renderers=[mainline_a], 
                mode='vline')

code2 = "source.set('selected', cb_data['index']);"
callback2 = CustomJS(args={'source': source2}, code=code2)
next_ht = HoverTool(
                tooltips=[("n", "@n2"),("d", "@z2"),("stat", "@y2"),('gd', "@k2")], 
                callback=callback2, 
                renderers=[next_a], 
                mode='vline')

p.add_tools(mainline_ht, next_ht)
show(p)
                        
